<a href="https://colab.research.google.com/github/melanieyes/flow-matching/blob/main/flow_matching_1point.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

## Data

In [ ]:
import torch
x_1 = torch.tensor([0.5])

## Flow Matching

In [ ]:
sig_min = 0

## Model

In [ ]:
class Flow(nn.Module):
  def __init__(self) -> None:
    super().__init__()

    self.linear = nn.Linear(2, 1)

  def forward(self,  t: torch.Tensor, x: torch.Tensor) -> torch.Tensor:
    t = t.unsqueeze(-1).float()
    x = torch.cat([x_t, t], dim=1)
    x = self.linear(x)
    x = nn.Tanh()(x)
    return x

## Training

In [ ]:
# Khởi tạo model
model = Flow()

# Chuẩn bị optimizer và loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
loss_fn = nn.MSELoss()

In [ ]:
losses = []
steps = 500

x_1 = x_1.unsqueeze(0)
for step in range(steps):
    optimizer.zero_grad()

    # Get x_0
    x_0 = torch.randn_like(x_1)

    # Get t
    t = torch.rand(1)

    # Target
    target = x_1 - (1 - sig_min) * x_0

    # Model predict
    x_t = (1 - (1-sig_min)*t) * x_0 + t*x_1
    pred = model(t, x_t)

    # Update
    loss = loss_fn(pred, target)
    loss.backward()
    optimizer.step()

    if step % 100 == 0:
        losses += [loss.detach()]
        print(f"Step: {step}, loss: {loss.item()}")

Step: 0, loss: 0.007723119575530291
Step: 100, loss: 0.574454665184021
Step: 200, loss: 0.003689428325742483
Step: 300, loss: 0.5774726867675781
Step: 400, loss: 0.024454666301608086


## Sampling

In [ ]:
steps = 10
with torch.no_grad():
    x_0 = torch.randn_like(x_1)
    t = torch.zeros(1)
    delta = 1/steps
    x_t = x_0
    list_x_t = [x_t]
    for i in range(steps):
        x_t = x_t + delta*model(t, x_t)
        t = t + delta
        list_x_t.append(x_t)


In [ ]:
list_x_t

[tensor([[-0.2825]]),
 tensor([[-0.1838]]),
 tensor([[-0.0861]]),
 tensor([[0.0099]]),
 tensor([[0.1031]]),
 tensor([[0.1917]]),
 tensor([[0.2737]]),
 tensor([[0.3471]]),
 tensor([[0.4108]]),
 tensor([[0.4649]]),
 tensor([[0.5110]])]